# Operation Short Bull's Bit-Bot

In [10]:
import robin_stocks.robinhood as r
import pandas as pd
import ta
import time
from dotenv import load_dotenv
import os
import numpy as np
import logging
import pyotp

logging.basicConfig(
    level=logging.INFO,  
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.StreamHandler()  
    ]
)

## Load in Secrets

In [11]:
# Load the .env file
load_dotenv("/Users/rosspingatore/operation_short_bull/operation_short_bull/secrets.env")

# Access the variables
api_key = os.getenv("username")
secret_key = os.getenv("password")
alpha_num_key = os.getenv("alpha_num_key")

## Define Helper Functions

In [12]:
def is_session_active():
    """Check if the Robinhood session is still active."""
    try:
        account_info = r.profiles.load_account_profile(info=None)
        return True if account_info else False
    except Exception as e:
        logging.error(f"Session check failed: {e}")
        return False
    
def get_btc_holdings(positions):
    """Extract Bitcoin holdings from the positions data."""
    btc_holdings = 0.0
    if positions:
        for position in positions:
            currency = position.get('currency', {})
            currency_code = currency.get('code') or currency.get('symbol')
            if currency_code == 'BTC':
                quantity = position.get('quantity_available') or position.get('quantity')
                if quantity:
                    btc_holdings = float(quantity)
                break
    return btc_holdings

def get_bitcoin_data(interval='5minute', span='day'):
    data = r.get_crypto_historicals(
        'BTC',
        interval=interval,
        span=span,
        bounds='24_7'
    )
    df = pd.DataFrame(data)
    df['begins_at'] = pd.to_datetime(df['begins_at'])
    df.set_index('begins_at', inplace=True)
    df['close_price'] = df['close_price'].astype(float)
    return df

def apply_sma_strategy(df):
    # Shorten the moving average windows for a more responsive strategy
    df['ma_short'] = df['close_price'].rolling(window=3).mean()
    df['ma_long'] = df['close_price'].rolling(window=7).mean()
    df['signal'] = 0
    # Fixing the chained assignment using .loc
    df.loc[df.index[3:], 'signal'] = np.where(
        df['ma_short'].iloc[3:] > df['ma_long'].iloc[3:], 1, 0
    )
    df['position'] = df['signal'].diff()
    
    # Log the latest moving averages and signal
    latest = df.iloc[-1]
    logging.info(f"Latest Close Price: {latest['close_price']}")
    logging.info(f"Latest Short MA (3): {latest['ma_short']}")
    logging.info(f"Latest Long MA (7): {latest['ma_long']}")
    logging.info(f"Latest Signal: {latest['signal']}")
    logging.info(f"Latest Position Change: {latest['position']}")
    
    return df

def apply_macd_strategy(df):
    # Adjust EMA spans for short-term sensitivity
    df['ema_short'] = df['close_price'].ewm(span=6, adjust=False).mean()
    df['ema_long'] = df['close_price'].ewm(span=13, adjust=False).mean()
    
    # Calculate MACD line and Signal line
    df['macd_line'] = df['ema_short'] - df['ema_long']
    df['signal_line'] = df['macd_line'].ewm(span=5, adjust=False).mean()
    
    # Generate trading signals
    df['signal'] = 0
    df['signal'] = np.where(df['macd_line'] > df['signal_line'], 1, 0)
    df['position'] = df['signal'].diff()
    
    # Log the latest MACD values and signal
    latest = df.iloc[-1]
    logging.info(f"Latest Close Price: {latest['close_price']}")
    logging.info(f"MACD Line: {latest['macd_line']}")
    logging.info(f"Signal Line: {latest['signal_line']}")
    logging.info(f"MACD Histogram: {latest['macd_line'] - latest['signal_line']}")
    logging.info(f"Trading Signal: {latest['position']}")
    
    return df

def execute_trades(df):
    try:
        if df.empty:
            logging.error("Empty DataFrame provided to execute_trades.")
            return

        latest = df.iloc[-1]
        btc_price = latest['close_price']

        # Check if latest data is valid
        if btc_price is None or np.isnan(btc_price):
            logging.error("Invalid BTC price data.")
            return

        # Get account cash balance
        account_profile = r.profiles.load_account_profile()
        cash_balance = float(account_profile.get('crypto_buying_power', 0))
        logging.info(f"Cash Balance: {cash_balance}")

        # Get BTC holdings
        positions = r.crypto.get_crypto_positions()
        btc_holdings = get_btc_holdings(positions)
        logging.info(f"BTC Holdings: {btc_holdings}")

        # Trade execution logic
        if latest['position'] == 1:
            # Buy signal
            buy_amount_usd = cash_balance * 0.10  # 10% of available cash
            if buy_amount_usd > 0:
                buy_quantity = buy_amount_usd / btc_price
                order_response = r.orders.order_buy_crypto_by_quantity('BTC', round(buy_quantity, 7))
                if 'id' in order_response:
                    logging.info(f"Bought {buy_quantity:.8f} BTC at {latest.name}")
                else:
                    logging.error(f"Buy order failed: {order_response}")
            else:
                logging.info(f"Insufficient cash to buy at {latest.name}.")
        elif latest['position'] == -1:
            # Sell signal
            sell_quantity = btc_holdings * 0.10  # 10% of BTC holdings
            if sell_quantity > 0:
                order_response = r.orders.order_sell_crypto_by_quantity('BTC', round(sell_quantity, 7))
                if 'id' in order_response:
                    logging.info(f"Sold {sell_quantity:.8f} BTC at {latest.name}")
                else:
                    logging.error(f"Sell order failed: {order_response}")
            else:
                logging.info(f"No BTC holdings to sell at {latest.name}.")
        else:
            logging.info(f"No trade executed at {latest.name}.")

    except Exception as e:
        logging.error(f"An error occurred during trade execution: {e}")

## Define Main

In [13]:
def main():
    # Initial login
    totp  = pyotp.TOTP(alpha_num_key).now()
    login = r.authentication.login(username=api_key, password=secret_key, mfa_code= totp)
    if login is None or 'access_token' not in login:
        logging.error("Failed to log in to Robinhood.")
        return

    logging.info("Successfully logged in to Robinhood.")

    while True:
        try:
            # Fetch and process Bitcoin data
            df = get_bitcoin_data()
            df = apply_macd_strategy(df)

            # Check if session is active
            if not is_session_active():
                logging.info("Session expired, logging in again.")
                totp  = pyotp.TOTP(alpha_num_key).now()
                login = r.authentication.login(username=api_key, password=secret_key, mfa_code= totp)
                if login is None or 'access_token' not in login:
                    logging.error("Failed to re-authenticate with Robinhood.")
                    continue  # Skip this iteration if re-authentication fails

            # Execute trades based on the strategy
            execute_trades(df)

        except Exception as e:
            logging.error(f"An error occurred in the main loop: {e}")


        # Wait for 5 minutes before the next iteration
        time.sleep(300)

    # Optional: Logout when the script is terminated (may not be reached in an infinite loop)

if __name__ == "__main__":
    main()

2024-11-30 12:11:11,988 - INFO - Successfully logged in to Robinhood.
2024-11-30 12:11:12,218 - INFO - Latest Close Price: 96941.505
2024-11-30 12:11:12,219 - INFO - MACD Line: 67.99948174758174
2024-11-30 12:11:12,219 - INFO - Signal Line: 47.973997750611026
2024-11-30 12:11:12,220 - INFO - MACD Histogram: 20.025483996970713
2024-11-30 12:11:12,220 - INFO - Trading Signal: 0.0
2024-11-30 12:11:12,490 - INFO - Cash Balance: 11.97
2024-11-30 12:11:12,604 - INFO - BTC Holdings: 7.49e-05
2024-11-30 12:11:12,606 - INFO - No trade executed at 2024-11-30 17:05:00+00:00.
2024-11-30 12:16:13,352 - INFO - Latest Close Price: 96903.56
2024-11-30 12:16:13,353 - INFO - MACD Line: 59.23983481377945
2024-11-30 12:16:13,354 - INFO - Signal Line: 51.77193663733344
2024-11-30 12:16:13,354 - INFO - MACD Histogram: 7.467898176446013
2024-11-30 12:16:13,355 - INFO - Trading Signal: 0.0
2024-11-30 12:16:13,638 - INFO - Cash Balance: 13.12
2024-11-30 12:16:13,752 - INFO - BTC Holdings: 7.49e-05
2024-11-30 1

KeyboardInterrupt: 

In [ ]:
r.authentication.logout()